# Werkloosheid en doden in de Verenigde Staten. 
In deze notebook zal ik twee datasets apart cleanen en vervolgens mergen zodat het gereed is om data analytics op uit te voeren.

In [1]:
import pandas as pd 

## Werkloosheid
Ik begin met het bewerken van de dataset die werkloosheid in de Verenigde Staten laat zien.

In [2]:
work = pd.read_csv('Unemployment in America Per US State.csv')
work.head(20)

,FIPS Code,State/Area,Year,Month,Total Civilian Non-Institutional Population in State/Area,Total Civilian Labor Force in State/Area,Percent (%) of State/Area's Population,Total Employment in State/Area,Percent (%) of Labor Force Employed in State/Area,Total Unemployment in State/Area,Percent (%) of Labor Force Unemployed in State/Area
0,1,Alabama,1976,1,"2,605,000","1,484,555",57.0,"1,386,023",53.2,"98,532",6.6
1,2,Alaska,1976,1,"232,000","160,183",69.0,"148,820",64.1,"11,363",7.1
2,4,Arizona,1976,1,"1,621,000","964,120",59.5,"865,871",53.4,"98,249",10.2
3,5,Arkansas,1976,1,"1,536,000","889,044",57.9,"824,395",53.7,"64,649",7.3
4,6,California,1976,1,"15,621,000","9,774,280",62.6,"8,875,685",56.8,"898,595",9.2
5,37,Los Angeles County,1976,1,"5,273,000","3,381,856",64.1,"3,081,806",58.4,"300,050",8.9
6,8,Colorado,1976,1,"1,832,000","1,230,966",67.2,"1,160,104",63.3,"70,862",5.8
7,9,Connecticut,1976,1,"2,248,000","1,442,847",64.2,"1,301,974",57.9,"140,873",9.8
8,10,Delaware,1976,1,"417,000","261,418",62.7,"240,543",57.7,"20,875",8.0
9,11,District of Columbia,1976,1,"520,000","334,691",64.4,"305,677",58.8,"29,014",8.7


Meteen drop ik de columns 'Month' (de andere dataset heeft de informatie alleen per jaar), 'FIPS Code' (de andere dataset heeft deze info niet, ik gebruik 'State/Area' om de staat te matchen) en de column 'Total Civilian Non-Institutional Population in State/Area' (Deze is simpelweg niet nodig). 

In [3]:
work.drop(['Month', 'FIPS Code', 'Total Civilian Non-Institutional Population in State/Area'], axis='columns', inplace=True)
len(work['State/Area'].unique())

53

Uit de output van de vorige cell kan je zien dat er 53 unieke waardes zijn voor de column 'State/Area'. Omdat ik alleen naar de states wil kijken, zal ik de "Area's" verwijderen. Ik weet dat de VS 50 states heeft, dus 3 area's zullen eruit moeten. 

In [4]:
work = work[~work['State/Area'].isin(['District of Columbia', 'Los Angeles County', 'New York city'])]

In de cell hierna voer ik de volgende stappen uit:
1. Column namen beknopter maken
2. De columns met hele getallen erin (labor_force, employement, unemployement) hebben komma's om duizenden aan te geven (bv. 1,642,981). Als ik het datatype van de column wil verandere geeft dit errors. Daarom verwijder ik de komma's.
3. Datatypes van de columns veranderen.
4. Omdat de rows in de dataset per maand waren, maar ik de maand heb verwijderd, heb ik nu 12 rows per state per jaar. Ik kies ervoor om het gemiddelde te nemen per jaar. 
5. Ik rond the getallen af op helen, en de percentages op 1 decimaal. 
6. Na de groupby operation zijn de state en year gebruikt voor een hierarcical index. Deze haal ik weg. 
7. De andere dataset bevat alleen gevallen tussen de jaren 1999 en 2013, dus die wil ik ook overhouden uit deze dataset. 
8. Laatste versie van de dataset

In [5]:
# Stap 1
work.rename({
    'State/Area': 'state',
    'Year': 'year',
    'Total Civilian Labor Force in State/Area': 'labor_force',
    "Percent (%) of State/Area's Population": 'labor_force_percent',
    'Total Employment in State/Area': 'employement',
    'Percent (%) of Labor Force Employed in State/Area': 'employement_percent',
    'Total Unemployment in State/Area': 'unemployement',
    'Percent (%) of Labor Force Unemployed in State/Area': 'unemployement_percent'
}, inplace=True, axis='columns')

# Stap 2
work.replace(',','', regex=True, inplace=True)

# Stap 3
work['labor_force'] = pd.to_numeric(work['labor_force'])
work['employement'] = pd.to_numeric(work['employement'])
work['unemployement'] = pd.to_numeric(work['unemployement'])

# Stap 4
work = work.groupby(['state', 'year']).mean()

# Stap 5
work[['labor_force', 'employement', 'unemployement']] = work[['labor_force', 'employement', 'unemployement']].astype(int)
work[['labor_force_percent', 'employement_percent', 'unemployement_percent']] = work[['labor_force_percent', 'employement_percent', 'unemployement_percent']].round(1)

# Stap 6
work.reset_index(inplace=True)

# Stap 7
work = work.loc[(work.year >= 1999) & (work.year <= 2013)]

# Stap 8
work.head(18)

,state,year,labor_force,labor_force_percent,employement,employement_percent,unemployement,unemployement_percent
23,Alabama,1999,2150126,64.0,2048691,60.9,101434,4.7
24,Alabama,2000,2147399,63.6,2047897,60.6,99502,4.6
25,Alabama,2001,2126942,62.7,2016673,59.4,110268,5.2
26,Alabama,2002,2113208,62.1,1988237,58.4,124971,5.9
27,Alabama,2003,2128286,62.2,2001012,58.5,127273,6.0
28,Alabama,2004,2138181,62.1,2018713,58.6,119467,5.6
29,Alabama,2005,2140116,61.5,2044937,58.8,95179,4.5
30,Alabama,2006,2169932,61.6,2083033,59.1,86899,4.0
31,Alabama,2007,2180868,61.3,2092450,58.8,88418,4.0
32,Alabama,2008,2177200,60.6,2049357,57.0,127843,5.9


## Doden
Nu zal ik werken aan de dataste die doodsoorzaken over de jaren heen laat zien.

In [6]:
deaths = pd.read_csv('leading_cause_death.csv')
deaths.head(10)

,YEAR,113_CAUSE_NAME,CAUSE_NAME,STATE,DEATHS,AADR
0,1999,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional Injuries,Alabama,2313,52.17
1,1999,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional Injuries,Alaska,294,55.91
2,1999,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional Injuries,Arizona,2214,44.79
3,1999,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional Injuries,Arkansas,1287,47.56
4,1999,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional Injuries,California,9198,28.71
5,1999,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional Injuries,Colorado,1519,38.98
6,1999,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional Injuries,Connecticut,1034,29.31
7,1999,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional Injuries,Delaware,267,35.25
8,1999,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional Injuries,District of Columbia,161,28.38
9,1999,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional Injuries,Florida,5961,35.73


Hier verwijder ik alleen de column '113_CAUSE_NAME'. Deze column is in principe hetzelfde als 'CAUSE_NAME' maar dan met officiele codes erbij die ik niet nodig heb. Daarnaast heet de 'STATE' column wat waardes die geen state zijn, dus deze verwijder ik. 

In [7]:
deaths.drop(['113_CAUSE_NAME'], axis='columns', inplace=True)
deaths = deaths[~deaths['STATE'].isin(['District of Columbia', 'United States'])]

In de cell hierna voer ik de volgende stappen uit:
1. De column namen beknopter maken
2. Stap 2 was erg onverwachts: Op kaggle (waar ik de dataset vandaan heb) staat bij de column statistieken dat er geen missende waardes waren. Daar werd ik als data engineer natuurlijk heel blij van. Totdat ik de column type naar int wou veranderen en de error message kreeg dat 'kan * niet naar numeric veranderen'; de eigenaar van de dataset heeft 'x' en '*' gebruikt om missende waardes aan te duiden in plaats van deze leeg te laten. Dit heeft kaggle niet opgepakt. Dus in deze stap verander ik deze missende waardes naar NaN. 
3. Ik verander de column naar numeriek zodat ik berekeningen kan maken met deze columns. 
4. Voor elke missende waarde kijk ik naar de state en de cause van de andere jaren om het gemiddelde te berekenen en deze in de vullen voor 'death'
5. De aadr heeft meer missende values, en omdat sommige states helemaal geen aadr hebben in de datasets kan ik de vorige methode niet uitvoeren. Daarom gebruik ik een methode die pandas zelf levert.

In [13]:
# Stap 1
deaths.rename({
    'YEAR': 'year',
    'CAUSE_NAME': 'cause',
    'STATE': 'state',
    'DEATHS': 'deaths',
    'AADR': 'aadr'
}, inplace=True, axis='columns')

# Stap 2
deaths.deaths = deaths.deaths.replace('x', pd.NA)
deaths.aadr = deaths.aadr.replace(['x', '*'], pd.NA)

# Stap 3
deaths.deaths = pd.to_numeric(deaths.deaths, errors='coerce')
deaths.aadr = pd.to_numeric(deaths.aadr, errors='coerce')

# Stap 4
grouped_data = deaths.groupby(['state', 'cause'])
mean_deaths = grouped_data['deaths'].transform(lambda x: x.fillna(x.mean()))
deaths.deaths = mean_deaths
deaths.deaths = deaths.deaths.astype(int)

# Stap 5
deaths['aadr'] = deaths['aadr'].fillna(method='ffill')
deaths.aadr = deaths.aadr.astype(int)

deaths.loc[deaths.state == 'Alabama'].head(10)

,year,cause,state,deaths,aadr
0,1999,Unintentional Injuries,Alabama,2313,52
53,1999,All Causes,Alabama,44806,1009
104,1999,Alzheimer's disease,Alabama,772,17
156,1999,Homicide,Alabama,438,9
208,1999,Stroke,Alabama,3148,71
260,1999,Chronic liver disease and cirrhosis,Alabama,412,9
312,1999,CLRD,Alabama,2179,48
364,1999,Diabetes,Alabama,1341,30
417,1999,Diseases of Heart,Alabama,13419,302
469,1999,Essential hypertension and hypertensive renal ...,Alabama,313,7


Nu heb ik een ander probleem. De doodsoorzaken zitten allemaal in 1 column. Maar om de datasets samen te voegen moet ik per state een uniek jaartal hebben, en niet 1 state met hetzelfde jaartal meerder keren voor elke doodsoorzaak. Daarom kies ik ervoor om voor elke doodsoorzaak een column te maken met behulp van de pivot_table functie. 

In [15]:
deaths_pivot_table = pd.pivot_table(deaths, index=['state', 'year'], values='deaths', columns=['cause'])
deaths_pivot_table.reset_index(inplace=True)
deaths_pivot_table.loc[deaths_pivot_table.state == 'Alabama'].head(20)

cause,state,year,All Causes,Alzheimer's disease,CLRD,Cancer,Chronic liver disease and cirrhosis,Diabetes,Diseases of Heart,Essential hypertension and hypertensive renal disease,Homicide,Influenza and pneumonia,Kidney Disease,Parkinson's disease,Pneumonitis due to solids and liquids,Septicemia,Stroke,Suicide,Unintentional Injuries
0,Alabama,1999,44806,772,2179,9506,412,1341,13419,313,438,1228,979,207,306,691,3148,555,2313
1,Alabama,2000,45062,895,2057,9807,407,1321,13406,291,444,1138,939,214,289,775,3183,583,2093
2,Alabama,2001,45316,1103,2204,9801,445,1344,13207,369,425,1105,974,239,290,771,2998,512,2211
3,Alabama,2002,46069,1189,2328,9698,425,1486,13197,343,416,1218,1032,232,323,765,3201,514,2228
4,Alabama,2003,46716,1268,2434,9812,440,1414,13150,402,434,1157,1062,265,309,854,3028,521,2179
5,Alabama,2004,46121,1385,2361,9756,477,1449,12774,413,369,994,1049,256,287,756,2986,541,2403
6,Alabama,2005,47090,1501,2382,9913,478,1429,12869,428,433,1011,1036,306,318,848,2952,535,2395
7,Alabama,2006,46977,1497,2309,9899,489,1453,12583,456,445,918,1104,291,273,836,2740,580,2506
8,Alabama,2007,46696,1517,2530,10025,505,1313,11926,475,480,898,1051,321,288,769,2747,592,2542
9,Alabama,2008,47707,1518,2733,10182,489,1386,12074,501,454,912,1107,348,256,899,2863,604,2509


Dit maakt weer een ander probleem. Zoals je kunt zien in de column hierboven is een hele belangrijke column die we eerst hadden weggelaten. Dit is de aadr column. De aadr column is erg belangrijk, omdat dit de column is waar we de verschillende staten mee moeten vergelijken. Definitie van aadr: 'The age-adjusted rates are rates that would have existed if the population under study had the same age distribution as the "standard" population. Therefore, they are summary measures adjusted for differences in age distributions.' (health.mo.gov). Alle states hebben andere bevolkingsdistributies. De aadr zorgt ervoor dat deze op gelijke grond komt zodat we deze kunnen vergelijken. Kortom: een belangrijke column. We zullen iets soortsgelijks doen voor de aadr column als we voor de causes column hebben gedaan. 


In [17]:
death_causes = deaths_pivot_table.columns

aadr_column_names = [cause + '_aadr' for cause in death_causes]
aadr_column_names = aadr_column_names[2:]
pivot_table_aadr = pd.pivot_table(data=deaths, index=['state', 'year'], values='aadr', columns='cause')

pivot_table_aadr.columns = aadr_column_names
pivot_table_aadr.reset_index(inplace=True)
pivot_table_aadr.head(20)


,state,year,All Causes_aadr,Alzheimer's disease_aadr,CLRD_aadr,Cancer_aadr,Chronic liver disease and cirrhosis_aadr,Diabetes_aadr,Diseases of Heart_aadr,Essential hypertension and hypertensive renal disease_aadr,Homicide_aadr,Influenza and pneumonia_aadr,Kidney Disease_aadr,Parkinson's disease_aadr,Pneumonitis due to solids and liquids_aadr,Septicemia_aadr,Stroke_aadr,Suicide_aadr,Unintentional Injuries_aadr
0,Alabama,1999,1009,17,48,210,9,30,302,7,9,28,22,4,7,15,71,12,52
1,Alabama,2000,1004,20,45,215,8,29,299,6,10,25,21,4,6,17,71,13,47
2,Alabama,2001,1002,25,48,213,9,29,292,8,9,24,21,5,6,17,66,11,49
3,Alabama,2002,1013,27,50,208,9,32,291,7,9,27,22,5,7,16,71,11,49
4,Alabama,2003,1020,28,52,208,9,30,288,8,9,25,23,5,6,18,66,11,48
5,Alabama,2004,998,31,50,204,9,30,278,9,8,22,22,5,6,16,65,11,52
6,Alabama,2005,1001,33,50,204,9,29,274,9,9,22,22,6,6,18,63,11,51
7,Alabama,2006,979,32,47,199,9,29,262,9,9,19,23,6,5,17,57,12,53
8,Alabama,2007,957,32,51,198,9,26,244,9,10,18,21,6,6,15,56,12,53
9,Alabama,2008,959,32,54,197,9,27,243,10,9,18,22,7,5,17,58,12,52


Nu zal ik de twee death datasets mergen.

In [18]:
deaths_final = pd.merge(deaths_pivot_table, pivot_table_aadr, on=['state', 'year'])
deaths_final.head(5)

,state,year,All Causes,Alzheimer's disease,CLRD,Cancer,Chronic liver disease and cirrhosis,Diabetes,Diseases of Heart,Essential hypertension and hypertensive renal disease,...,Essential hypertension and hypertensive renal disease_aadr,Homicide_aadr,Influenza and pneumonia_aadr,Kidney Disease_aadr,Parkinson's disease_aadr,Pneumonitis due to solids and liquids_aadr,Septicemia_aadr,Stroke_aadr,Suicide_aadr,Unintentional Injuries_aadr
0,Alabama,1999,44806,772,2179,9506,412,1341,13419,313,...,7,9,28,22,4,7,15,71,12,52
1,Alabama,2000,45062,895,2057,9807,407,1321,13406,291,...,6,10,25,21,4,6,17,71,13,47
2,Alabama,2001,45316,1103,2204,9801,445,1344,13207,369,...,8,9,24,21,5,6,17,66,11,49
3,Alabama,2002,46069,1189,2328,9698,425,1486,13197,343,...,7,9,27,22,5,7,16,71,11,49
4,Alabama,2003,46716,1268,2434,9812,440,1414,13150,402,...,8,9,25,23,5,6,18,66,11,48


En nu zal ik de work en death datasets samenvoegen. Nu heb ik 1 bewerkte dataset die ik export als csv, waar ik in Power BI verder mee zal werken.

In [19]:
final_dataset = pd.merge(deaths_final, work, on=['state', 'year'])
final_dataset.to_csv('death_work.csv')
final_dataset.head(5)

,state,year,All Causes,Alzheimer's disease,CLRD,Cancer,Chronic liver disease and cirrhosis,Diabetes,Diseases of Heart,Essential hypertension and hypertensive renal disease,...,Septicemia_aadr,Stroke_aadr,Suicide_aadr,Unintentional Injuries_aadr,labor_force,labor_force_percent,employement,employement_percent,unemployement,unemployement_percent
0,Alabama,1999,44806,772,2179,9506,412,1341,13419,313,...,15,71,12,52,2150126,64.0,2048691,60.9,101434,4.7
1,Alabama,2000,45062,895,2057,9807,407,1321,13406,291,...,17,71,13,47,2147399,63.6,2047897,60.6,99502,4.6
2,Alabama,2001,45316,1103,2204,9801,445,1344,13207,369,...,17,66,11,49,2126942,62.7,2016673,59.4,110268,5.2
3,Alabama,2002,46069,1189,2328,9698,425,1486,13197,343,...,16,71,11,49,2113208,62.1,1988237,58.4,124971,5.9
4,Alabama,2003,46716,1268,2434,9812,440,1414,13150,402,...,18,66,11,48,2128286,62.2,2001012,58.5,127273,6.0
